In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import sda_grid_search
from keras.models import load_model


import os
import time
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

#variables para guardar los resultados
tipo = pruebas[4]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

Using Theano backend.
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Dataset de Twitter

In [3]:
print tipo
print dataset_name
print data_path
print dims

sda
twitter
data
1000


In [4]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/twitter/sda/me2\_[dominio\_fuente]_[dominio\_objetivo].pkl

### Creación de modelos de adaptación.

In [5]:
def split_src_tgt(X_src, X_tgt, test_size=0.2):
    # se divide el dataset para los datos de entrenamiento y validacion del SDA
    X_tr_src, X_val_src, _, _ = train_test_split(X_src, np.zeros(X_src.shape[0]), test_size=0.2, random_state=42)
    
    X_tr_tgt, X_val_tgt, _, _ = train_test_split(X_tgt, np.zeros(X_tgt.shape[0]), test_size=0.2, random_state=42)
    
    X_train = np.concatenate((X_tr_src, X_tr_tgt))
    X_val = np.concatenate((X_val_src, X_val_tgt))
    
    return X_train, X_val

In [6]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_models_paths.pkl')

parameters = {
    #'noises': [0.3 , 0.5, 0.8],
    #'layers': [[int(dims/2)], [int(dims/2), int(dims/4)]],
    #'epochs': [50, 25],
    'noises': [0.3],
    'layers': [[int(dims/2)]],
    'epochs': [50],
}

i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "%d) %s - %s" % (i, src, tgt)
            X_src = np.asarray(labeled[src]['X_tr'].todense())
            y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

            X_tgt = np.asarray(labeled[tgt]['X_tr'].todense())
            
            X_train, X_val = split_src_tgt(X_src, X_tgt, test_size=0.2)
            
            best_autoencoder, best_encoder, score = sda_grid_search(X_train, X_val, X_src, y_src, parameters, n_jobs=4)
            print "\tMejor modelo: %.3f" % score
            
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix_ae = "me2_ae_" + src + "_" + tgt + ".h5"
            prefix_e = "me2_e_" + src + "_" + tgt + ".h5"
            
            best_ae_save_path = os.path.join(folder_path, prefix_ae)
            best_e_save_path = os.path.join(folder_path, prefix_e)
            
            print "\tGuardando autoencoder en %s" % best_ae_save_path
            best_autoencoder.save(best_ae_save_path)
                
            print "\tGuardando encoder en %s\n" % best_e_save_path
            best_encoder.save(best_e_save_path)

            i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
1) thevoice - general
	pr: 0.300 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	0.875

	Mejor modelo: 0.875
	Guardando autoencoder en models/twitter/sda/me2_ae_thevoice_general.h5
	Guardando encoder en models/twitter/sda/me2_e_thevoice_general.h5

2) thevoice - rio2016
	pr: 0.300 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	0.858

	Mejor modelo: 0.858
	Guardando autoencoder en models/twitter/sda/me2_ae_thevoice_rio2016.h5
	Guardando encoder en models/twitter/sda/me2_e_thevoice_rio2016.h5

3) general - thevoice
	pr: 0.300 - epochs: 50 - layers: [500]
	Entrenando autoencoder...


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your d

	0.712

	Mejor modelo: 0.712
	Guardando autoencoder en models/twitter/sda/me2_ae_general_thevoice.h5
	Guardando encoder en models/twitter/sda/me2_e_general_thevoice.h5

4) general - rio2016
	pr: 0.300 - epochs: 50 - layers: [500]
	Entrenando autoencoder...


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your d

	0.717

	Mejor modelo: 0.717
	Guardando autoencoder en models/twitter/sda/me2_ae_general_rio2016.h5
	Guardando encoder en models/twitter/sda/me2_e_general_rio2016.h5

5) rio2016 - thevoice
	pr: 0.300 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	0.864

	Mejor modelo: 0.864
	Guardando autoencoder en models/twitter/sda/me2_ae_rio2016_thevoice.h5
	Guardando encoder en models/twitter/sda/me2_e_rio2016_thevoice.h5

6) rio2016 - general
	pr: 0.300 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	0.879

	Mejor modelo: 0.879
	Guardando autoencoder en models/twitter/sda/me2_ae_rio2016_general.h5
	Guardando encoder en models/twitter/sda/me2_e_rio2016_general.h5


Adaptaciones completadas


## Pruebas de clasificación

In [7]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### SDA ####
            #############
            # se adaptan los dominios usando SDA
            print "Adaptando dominios..."
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix_e = "me2_e_" + src + "_" + tgt + ".h5"
            best_e_save_path = os.path.join(folder_path, prefix_e)
            
            best_encoder = load_model(best_e_save_path)
            
            
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            X_tr_a = best_encoder.predict(X_tr)
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            X_ts_a = best_encoder.predict(X_ts)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
            t_error = 1-clf.score(X_ts_a, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "\nPruebas completadas."

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/keras/models.py:240: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your d

Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your d

Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [9]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,SDA,t->g,thevoice,general,46.524064,43.331543,-3.192521
2,SDA,t->r,thevoice,rio2016,58.333333,18.874773,-39.458560
3,SDA,g->t,general,thevoice,50.000000,28.669508,-21.330492
4,SDA,g->r,general,rio2016,58.333333,21.460980,-36.872353
5,SDA,r->t,rio2016,thevoice,50.000000,22.301136,-27.698864
6,SDA,r->g,rio2016,general,46.524064,42.597565,-3.926499


In [10]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/sda/me2_1000.csv
Resultados guardados.
